In [ ]:
import statsmodels.api as sm
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
import numpy as np

color = [x / 255. for x in [31,119,189]]
alpha = 153/255.

In [ ]:
df = pd.read_csv("fig_data/SuppFig2.csv")

#print(list(cnn_gdi_predictions))
mappings = {
    "FootProg_meanStance": "Mean foot progression angle at stance",
    "HipRot_mean": "Mean hip adduction",
}


def compare_with_residuals(dependent, var, letter, order=2):
    cnn_gdi_predictions = df
    cnn_gdi_predictions["%s_pred_residuals" % dependent] = cnn_gdi_predictions["%s_pred_corrected" % dependent] - cnn_gdi_predictions[dependent]

    corr = cnn_gdi_predictions.corr()['%s_pred_corrected' % dependent][var]

    sns.set(rc={'figure.figsize':(5.3,5.3),'axes.titlesize': 14})
    sns.set_style("ticks")

    ax = sns.regplot(cnn_gdi_predictions[var],
                     cnn_gdi_predictions['%s_pred_residuals' % dependent],
                     order=order,
                     color=color, 
                     scatter_kws={'alpha':alpha})
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    var_sq = "%s_sq" % var
    cnn_gdi_predictions[var_sq] = cnn_gdi_predictions[var]**2
    mod = sm.OLS(cnn_gdi_predictions['%s_pred_residuals' % dependent], cnn_gdi_predictions[[var, var_sq][:order]],missing='drop')
    fii = mod.fit()
    print(fii.summary2())

    plt.text(-0.13, 1.04, letter,
            horizontalalignment='center',verticalalignment='center',
            transform=ax.transAxes,size=14)

    plt.text(0.85, 0.14, r"r $= %.2f$" % math.sqrt(fii.rsquared_adj), horizontalalignment='center',verticalalignment='center', transform=ax.transAxes)
    plt.text(0.85, 0.07, r"p $< 10^{-4}$", horizontalalignment='center',verticalalignment='center', transform=ax.transAxes)
    nl = ""
    if letter=="a":
        nl="\n"
    plt.title("%s %svs residual %s" % (mappings.get(var, var), nl, dependent))
    plt.xlabel(mappings.get(var, var) + " [degrees]")
    plt.ylabel("Residual %s" % dependent)
    plt.savefig("output/Supplement-Figure-2%s.pdf" % letter,bbox_inches="tight",pad_inches=0.1)
    plt.show()

    return fii

compare_with_residuals("GDI", "HipRot_mean", "b", 2)
fii = compare_with_residuals("GDI", "FootProg_meanStance", "a", 2)